# 量子位相推定 (Quantum Phase Estimation)
ハミルトニアン$H$と固有状態$\mid \psi \rangle$が準備されたとき、その固有値$\lambda = e^{-i\alpha}$の位相$\alpha$を求めることができる量子位相推定を確認します。

$$
H \mid \psi \rangle = e^{-i\alpha} \mid \psi \rangle
$$

## 量子位相推定の量子回路
量子位相推定アルゴリズムの全体像は下記の通りです。

1. 固有ベクトルとしての量子状態の準備  
2. 固有値を量子状態として取り出す（位相キックバック）  
3. 固有値の量子状態をビットに変換（量子フーリエ変換）  

の３つからなります。


```
        step2                 step3
      
|0> ----H----------- -*-------iQFT---
                      |
|0> ----H--------*-- -|-------iQFT---
|0> ----H-----*--|-- -|-------iQFT---
|0> ----H--*--|--|-- -|-------iQFT---
           |  |  |    |
           |  |  |    |
|ψ> -------U1-U2-U4- -U2n------------
step1
```

## 位相キックバック
基本的には固有値固有ベクトル問題を利用し、ハミルトニアンHを導入したユニタリ回路を制御付きにすることにより、固有ベクトル$\mid \psi \rangle$に対応する固有値$e^{2\pi i \phi}$をコントロールビットの方の$\mid 1 \rangle$状態にかけることができます。

対応するハミルトニアンに対する固有値固有ベクトルは、

$$
U\mid \psi \rangle = e^{2\pi i \phi} \mid \psi \rangle
$$

で書くことができます。量子状態$\mid \psi \rangle$は準備されてますので、ハミルトニアンを導入したユニタリ回路Uを準備することで、位相を移すことができます。

まず初期状態からアダマールゲートを使って結果を取り出したい方の量子ビットを重ね合わせ状態にします。

$$
\mid 0 \rangle \mid \psi \rangle \rightarrow \frac{\mid 0\rangle + \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle = \frac{\mid 0\rangle \mid \psi \rangle + \mid 1 \rangle \mid \psi \rangle}{\sqrt{2}}
$$

次に制御付きのユニタリ回路（ハミルトニアンに対応した形）を導入することで、制御付きなので量子ビットが$\mid 1 \rangle$の方にだけユニタリがかかります。

$$
\frac{\mid 0\rangle \mid \psi \rangle + \mid 1 \rangle U \mid \psi \rangle}{\sqrt{2}}
$$

そして、これは固有値に対応しているので、

$$
\frac{\mid 0\rangle \mid \psi \rangle + \mid 1 \rangle e^{2\pi i \phi} \mid \psi \rangle}{\sqrt{2}} = \frac{\mid 0\rangle \mid \psi \rangle +  e^{2\pi i \phi} \mid 1 \rangle \mid \psi \rangle}{\sqrt{2}} = \frac{\mid 0\rangle +  e^{2\pi i \phi} \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle
$$

このようになりました。これによって、固有値をかけることができます。あとは、

$$
\frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{i2\pi k\phi}\mid k \rangle
$$

のように、取り出したい桁に対応するkを導入すれば良いですが、これは回転角に対応しており、固有値をk回かけるということをすると、自然と対応することができます。つまり$U^k$のように同じユニタリ操作をk回実行すれば良いことになります。ということで、k回Controlled-Unitary操作を行うことで求めることができます。

$$
\frac{\mid 0\rangle +  U^k \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle = \frac{\mid 0\rangle +  e^{2\pi i k \phi} \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle
$$

これにより、対応する桁kに対してk回の制御付きユニタリゲートをかけることで実行が終了します。

## 量子フーリエ変換
量子フーリエ変換は入力を01ビットの配列から、その配列に対応する位相をもつ量子状態の形に変形をすることができます。量子フーリエ変換の逆回路である逆量子フーリエ変換を利用することで、上記位相キックバックで移した位相をビット列で書き出すことができます。

$$
QFT:\mid x \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1} \omega_n^{xk}\mid k\rangle
$$

$\omega_n = e^{\frac{2\pi i}{N}}$とすると、

$$
{F_N= 
\frac{1}{\sqrt{N}} 
\left[ 
 \begin{array}{rrrr} 
  1 & 1 & 1 & \cdots &1\\ 
  1 & \omega_n&\omega_n^2&\cdots&\omega_n^{N-1}\\ 
  1 & \omega_n^2&\omega_n^4&\cdots&\omega_n^{2(N-1)}\\ 
  1 & \omega_n^3&\omega_n^6&\cdots&\omega_n^{3(N-1)}\\ 
  \vdots&\vdots&\vdots&&\vdots\\ 
  1 & \omega_n^{N-1}&\omega_n^{2(N-1)}&\cdots&\omega_n^{(N-1)(N-1)} 
 \end{array} 
\right]
}
$$

のようになります。$x_1$から$x_n$のビットを入力すると、それに対応した位相が量子状態の形で出力されます。

$$
QFT(\mid x_1,x_2,…,x_n \rangle) = \frac{1}{\sqrt{N}}(\mid 0 \rangle + e^{2\pi i [0.x_n]} \mid 1 \rangle) \otimes … \otimes (\mid 0 \rangle + e^{2\pi i [0.x_1x_2…x_n]} \mid 1 \rangle)
$$

出力された量子状態はそれぞれの量子ビットは位相の角度の精度に対応された形に出力されますが、それぞれの位相に対応する係数は量子ビットから見ると絶対値はすべて1なので、測定しただけでは0と1が完全に50%ずつ出てしまうのが特徴です。

$$[0.x_1x_2…] = \frac{x_1}{2}+\frac{x_2}{2^2}+…$$

位相は小数点の形で取り出されます。

量子位相推定では、作った状態ベクトル$\mid \psi \rangle$の位相である$\lambda = e^{-i\alpha}$を求めたい場合、状態ベクトルを上記の量子フーリエ変換の形に対応させた形で変形させ、それの状態ベクトルを再度逆量子フーリエ変換にかけることで、任意の精度で位相を小数点表記で取り出すことができます。

## Zゲートの位相を推定する
例題をやってみます。まずはハミルトニアンとしてZゲートを用意してみます。

$$
Z = \begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}
$$

まずは手計算で答えを確認してみると、固有値は、

$$
det\begin{bmatrix}
1-\lambda&0\\
0&-1-\lambda
\end{bmatrix}
$$

を計算して、$\lambda = 1,-1$で、固有ベクトルは、

$$
\begin{bmatrix}
1\\
0
\end{bmatrix},
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

こうなるはずです。早速確かめてみます。

## blueqatのインストール
pip経由でインストールを行います。

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 回路の全体概要
今回の回路の全体概要です。

```
|0> ----H--*--iQFT--M
           |
|0> -------Z--------
```

まず２量子ビット準備します。0番目、1番目と名前をつけます。どちらも量子ビットは0からスタートします。

１、今回は最初っから量子状態を準備します。
２、次に0番目の量子ビットを重ね合わせ状態にして、CZゲートで制御付きのハミルトニアンを実行して位相を0番目の量子ビットにキックバックします。
３、最後に出来上がった量子状態から位相をビット表記で取り出すための逆量子フーリエ変換を実行し、測定して終わりです。

## 量子状態の準備
固有ベクトルを|0>にして計算しますので、何もしないで大丈夫そうです。

## 位相キックバック
次に量子状態から位相をキックバックします。

```
|0> --H--*--iQFT--M
         |
|0> -----Z--------
```

量子状態が準備できました。0番目の量子ビットを重ね合わせにして、量子状態を固有ベクトルとして、固有値を0番目の方に情報としてうつします。

In [3]:
from blueqat import Circuit

Circuit().h[0].cz[0,1].h[0].m[:].run(shots=100)

Counter({'00': 100})

となり、

$$
\phi = 0.0
$$

求める固有値は、

$$
e^{2\pi i \phi} = e^{2\pi i *0} = e^0 = 1
$$

となりました。

## 固有状態を|1>に
最初の量子状態を|1>にしてみます。

```
|0> --H--*--iQFT--M
         |
|0> --X--Z--------
```

In [6]:
Circuit().x[1].h[0].cz[0,1].h[0].m[:].run(shots=100)

Counter({'11': 100})

こちらは11となりました。

$$
\phi = 1/2 = 0.5
$$

$$
e^{2\pi i *0.5} = -1
$$

となりました。

## Xゲート
Xゲートは、

$$
X = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
$$

となります。固有ベクトルとしてまずは、

$$
\mid \psi \rangle = 
\begin{bmatrix}
1\\
1
\end{bmatrix}
$$

を考えると、

```
|0> --H--*--H--M
         |
|0> --H--X-----
```

を実行してみます。

In [7]:
Circuit(2).h[:].cx[0,1].h[0].m[0].run(shots=100)

Counter({'00': 100})

こちらは、$\phi = 0$となり、

$$
\lambda = e^0=1
$$

となりました。続いて固有ベクトルが、

$$
\mid \psi \rangle = 
\begin{bmatrix}
1\\
-1
\end{bmatrix}
$$

を考えます。

```
|0> --H---*--H--M
          |
|0> --HZ--X-----
```

HZゲートで量子状態を準備します。

In [8]:
Circuit(2).h[:].z[1].cx[0,1].h[0].m[0].run(shots=100)

Counter({'10': 100})

これより、$\phi = 0.5$となり、

$$
\lambda = e^{2\pi i *0.5}=-1
$$

となりました。